# GNN

> 1. Convolutional 
>> - Spatial-based
>>
>> |Aggregation| Method|
>>|---|---|
>>|Sum | NN4G|
>>|Mean | DCNN,DGC, GraphSAGE|
>>|Weighted Sum| MoNET, **GAT**, GIN|
>>|LSTM | GraphSAGE|
>>|Max Pooling| GraphSAGE|\
>>
>> Readout: For multiple layers(L), Xi = agg(Li), y = Sum of Wi * Xi
>> - Spectral-based: \
"Power" of signal variation between nodes
 1/2 * EEWi,j (f(vi)-f(vj))^2 
>>> ChebNet -> ** GCN ** -> HyperGCN\
>>> Discrete time Fourier basis: 频率越大，相邻两点之间信号变化量越大
> 2. Graph signal processing 

## Application
- GIN
- GatedGCN (GCN+GAT)

import torch
import torch.nn as nn
from torch_geometric.nn import GATConv, GCNConv, GINConv


class GAT(nn.Module):
    def __init__(self, config):
        super().__init__()
        # Properties
        self.config = config
        self.num_heads = config.num_heads
        self.input_dim = config.input_dim
        self.hidden_dim = config.hidden_dim
        self.output_dim = config.output_dim
        self.dropout = config.dropout

        # Layers
        self.conv1 = GATConv(self.input_dim, self.hidden_dim, heads=self.num_heads)
        self.relu1 = nn.ReLU()
        self.conv2 = GATConv(self.num_heads * self.hidden_dim, self.hidden_dim, heads=self.num_heads)
        self.relu2 = nn.ReLU()
        self.classifier = nn.Sequential(
            nn.Linear(self.num_heads * self.hidden_dim, self.hidden_dim),
            nn.Dropout(self.dropout),
            nn.Linear(self.hidden_dim, self.output_dim),
            nn.Sigmoid()
        )

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = self.relu1(x)
        x = self.conv2(x, edge_index)
        x = self.relu2(x)
        x = self.classifier(x)
        return x
    

class GCN(nn.Module):
    def __init__(self, config):
        super(GCN, self).__init__()
        # Properties
        self.config = config
        self.input_dim = config.input_dim
        self.hidden_dim = config.hidden_dim
        self.output_dim = config.output_dim

        # Layers
        self.conv1 = GCNConv(self.input_dim, self.hidden_dim)
        self.conv2 = GCNConv(self.hidden_dim, 2)
        self.classifier = nn.Linear(2, self.output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        h = self.conv1(x, edge_index)
        h = h.tanh()
        h = self.conv2(h, edge_index)
        embeddings = h.tanh()
        out = self.classifier(embeddings)
        return torch.sigmoid(out)


class GIN(nn.Module):
    def __init__(self, config):
        super(GIN, self).__init__()
        # Properties
        self.config = config
        self.input_dim = config.input_dim
        self.hidden_dim = config.hidden_dim
        self.output_dim = config.output_dim

        # Layers
        self.conv1 = GINConv(
            nn.Sequential(
                nn.Linear(self.input_dim, self.hidden_dim),
                nn.ReLU(),
                nn.Linear(self.hidden_dim, self.hidden_dim),
                nn.ReLU(),
            )
        )
        self.conv2 = GINConv(nn.Sequential(
            nn.Linear(self.hidden_dim, self.hidden_dim),
            nn.ReLU(),
            nn.Linear(self.hidden_dim, self.hidden_dim),
            nn.ReLU()
        ))
        self.fc = nn.Linear(self.hidden_dim, self.output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = self.fc(x)
        return torch.sigmoid(x)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
#import networkx as nx
import torch
import torch.nn as nn
import torch.nn.functional as F

c:\ProgramData\Miniconda3\envs\fj\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
df = pd.read_csv("../../Data/creditcard.csv")
del df["Time"]

zero = df[df["Class"] == 0].sample(frac = 1, random_state= 1234)
one = df[df["Class"] != 0]

train_df = zero[:int(round(zero.shape[0])*0.8)]
test_df = zero[int(round(zero.shape[0])*0.8):].append(one)

df["Amount"] = np.log(df["Amount"]+1).apply(lambda x: math.ceil(x * 10000)/10000)

gc.collect()